In [ ]:
import requests
import time
import os
import pandas as pd
import re


def trans_date(v_timestamp):
    timeArray = time.localtime(v_timestamp)
    otherStyleTime = time.strftime("%Y-%m-%d", timeArray)
    print(otherStyleTime)
    return otherStyleTime


def get_search(v_keyword, v_max_page, v_out_file):
    for page in range(1, v_max_page + 1):
        print('开始爬取第{}页'.format(page))
        url = 'https://api.bilibili.com/x/web-interface/search/type'
        headers = {'accept': 'application/json, text/plain, */*',
                   'accept-encoding': 'gzip, deflate, br',
                   'accept-language': 'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7',
                   'cookie': "buvid3=EAD2B70B-8739-A7EC-3C83-B444C2A7842D40993infoc; b_nut=1688904540; CURRENT_FNVAL=4048; _uuid=839239AC-10A103-35105-943F-8CBA67E6863241028infoc; buvid4=9D0D4BA7-2233-10C2-3E39-0829876A177019287-023050618-odT3LzXWZnL3oPcjsh4EkQ%3D%3D; rpdid=|(J|Y)RllYmk0J'uY))m|RlRR; fingerprint=909c249cd3d78e64b4c66e3e5a90a460; buvid_fp_plain=undefined; header_theme_version=CLOSE; home_feed_column=5; browser_resolution=1920-963; PVID=1; buvid_fp=06282d96620fafb14f121209f007726f; innersign=0; b_lsid=7107CE854_18A133CA4DB; SESSDATA=01224790%2C1708092012%2C17f8e%2A81JarWakqsFd1-csCr8XITIJdrkpBYvdfo-06oP1P4FZcNGcHcjhzUG0kKekfiNTjzhVbu1QAAAwA; bili_jct=601c7da3a64ccae45866a00910ca3a1f; DedeUserID=3537121411337103; DedeUserID__ckMd5=2ca39f6470404094; sid=5rya6tex; bp_video_offset_3537121411337103=828080432103817381",
                   'origin': 'https://search.bilibili.com',
                   'referer': 'https://search.bilibili.com/all?keyword={}&from_source=webtop_search&spm_id_from=333.1007&search_source=3&page=3&o=48'.format(
                      v_keyword),
                   'sec-ch-ua': '"Chromium";v="116", "Not)A;Brand";v="24", "Google Chrome";v="116"',
                   'sec-ch-ua-mobile': '?0',
                   'sec-ch-ua-platform': '"Windows"',
                   'sec-fetch-dest':'empty',
                   'sec-fetch-mode':'cors',
                   'sec-fetch-site':'same-site',
                   'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36'}
        params = {
            '__refresh__': 'true',
            '_extra': '',
            'context':  '',
            'page': page,
            'page_size': 36,
            'from_source': '',
            'from_spmid': '333.337',
            'platform': 'pc',
            'highlight': '1',
            'single_column': '0',
            'keyword': v_keyword,
            'qv_id': '5b43iVihS7VLlNOZ5mSEEiMee6rKG9jK',
            'category_id': '',
            'search_type': 'video',
            'dynamic_offset': 30 * (page - 1),
            'preload': 'true',
            'com2co': 'true',
        }
        r = requests.get(url, headers=headers, params=params)
        print(r.status_code)
        j_data = r.json()
        data_list = j_data['data']['result']
        print('数据长度： ', len(data_list))
        title_list = []
        author_list = []
        bvid_list = []
        create_time = []
        duration_list = []
        danmaku_list = []
        like_list = []
        play_list = []
        favorites_list = []
        typename_list = []
        tag_list = []
        desc_list = []
        for data in data_list:
            title = re.compile(r'<[^>]+>', re.S).sub('', data['title'])
            print('视频标题: ' + title)
            title_list.append(title)
            author_list.append(data['author'])
            bvid_list.append(data['bvid'])
            create_time.append(trans_date(v_timestamp=data['pubdate']))
            duration_list.append(data['duration'])
            danmaku_list.append(data['danmaku'])
            like_list.append(data['like'])
            play_list.append(data['play'])
            favorites_list.append(data['favorites'])
            typename_list.append(data['typename'])
            tag_list.append(data['tag'])
            desc_list.append(data['description'])
        df = pd.DataFrame(
            {
                '页码': [page] * len(title_list),
                '视频标题': title_list,
                '视频作者': author_list,
                '视频编号': bvid_list,
                '创建时间': create_time,
                '视频时间': duration_list,
                '弹幕数' : danmaku_list,
                '点赞数': like_list,
                '播放数': play_list,
                '收藏数' : favorites_list,
                '分区类型': typename_list,
                '标签': tag_list,
                '视频描述': desc_list,
            }
        )
        csv_header = ['页码','视频标题','视频作者','视频编号','创建时间','视频时长',
                      '弹幕数', '点赞数','播放数','收藏数','分区类型','标签','视频描述']
        if os.path.exists(v_out_file):
            header = None
        else:
            header = csv_header
        df.to_csv(v_out_file, encoding='utf_8_sig', mode='a+', index=False, header=header)


if __name__ == '__main__':
    now = time.strftime('%Y%m%d', time.localtime())
    search_keyword = '孔乙己文学'
    max_page = 28
    result_file = 'B站视频_{}_前{}页_{}.csv'.format(search_keyword, max_page, now)
    get_search(v_keyword=search_keyword.encode('utf-8'),
               v_max_page=max_page,
               v_out_file=result_file
               )

开始爬取第1页
200
数据长度：  36
视频标题: 【围炉夜话】震撼人心！没想到我们的初中课本上，有这么牛逼的课文。（谈鲁迅8）
2020-12-14
视频标题: 【话剧】孔乙己 完整版
2018-05-30
视频标题: 凉 乙 己
2022-12-12
视频标题: 不当白领去做体力活？我替你们体验过了...
2023-03-18
视频标题: 谢谢，但我不是孔乙己！就算是，也不需要别人傲慢的评价我。
2023-03-24
视频标题: 孔乙己的长衫，吐槽可以，别太当真！
2023-03-18
视频标题: 轻音少女 X 孔乙己
2020-08-27
视频标题: 央视网评“孔乙己文学”：孔乙己之所以陷入生活困境不是因为读过书，而是放不下读书人的架子
2023-03-17
视频标题: 【浮夸】孔乙己的内心独白
2016-10-06
视频标题: 鲁迅在今天会成为公知吗？看懂《孔乙己》你就知道是不可能的【大师计划·倪文尖02】
2021-03-04
视频标题: 126年历史！鲁迅笔下的咸亨酒店，孔乙己同款茴香豆！
2021-02-08
视频标题: 孔乙己那个脱不掉的长衫不是学历，而是……
2023-02-26
视频标题: 【国产/剧情】孔乙己 （1981）【鲁迅全集·短篇小说改编】
2016-01-23
视频标题: 鲁 迅 十 字 军
2021-05-01
视频标题: 国家线出了，我们每个人真的都需要考研吗？孔乙己的长衫你需要吗？【一起上岸吧！】
2023-03-14
视频标题: Vol.94 孔乙己的长衫 谈“孔乙己文学”
2023-03-17
视频标题: 【孔乙己】谈鲁迅，孔乙己神在哪里？我们不止是孔乙己！
2021-07-31
视频标题: 从「小王子」到「孔乙己」，理想主义者注定死路一条吗？【心缘驿马#2】
2022-03-31
视频标题: 用孔乙己文学自嘲的年轻人，自怨自艾不如付诸行动
2023-02-28
视频标题: 讲个故事《孔乙己》
2023-01-12
视频标题: 2022年是孔乙己欠酒账100周年
2022-04-29
视频标题: 深挖到骨子里的“国民之弊”——深读《阿Q正传》《药》《孔乙己》｜【正讲鲁迅】02期
2022-08-13
视频标题: 孔乙己的时代真如央视网所说一去不复返了吗？
2023-03-17
视频标题: 【FF14】怎么可以不干饭！！！